In [1]:
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd

#data scaling
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split , KFold, cross_val_score, LeaveOneOut
from sklearn.naive_bayes import MultinomialNB

import spacy
import re

In [2]:
# preprocessing
data = pd.read_csv("TrainingSet1.csv")

print( data )

#temporaire pour moins de données
#data = data.head()

intentions = data["intention"].unique()
data["intention"] = data["intention"].replace({"arrosage" : np.where( intentions == 'arrosage'),
                                              "soleil": np.where( intentions == 'soleil'), 
                                              "tailler" : np.where( intentions == 'tailler'),
                                              "temperature" : np.where( intentions == 'temperature'),
                                              "cadeaux" : np.where( intentions == 'cadeaux'),
                                              "varietes" : np.where( intentions == 'varietes'),
                                              "entretien" : np.where( intentions == 'entretien'),
                                               "utilisation" : np.where( intentions == 'utilisation'),
                                               "planter" : np.where( intentions == 'planter'),
                                               "maladies" : np.where( intentions == 'maladies')
                                              })

                                              sentence    intention
0         Est ce que j'ai besoin d'arroser ma plante ?     arrosage
1                    Quand dois-je arroser ma plante ?     arrosage
2                          Dois-je arroser ma plante ?     arrosage
3                  Comment puis-je arroser ma plante ?     arrosage
4               Ma plante a besoin de beaucoup d'eau ?     arrosage
5    De quelle quantité d'eau ma plante a t'elle be...     arrosage
6                         J'aimerais arroser ma plante     arrosage
7                                     Ma plante a soif     arrosage
8                        Je pense que ma plante a soif     arrosage
9    Ma plante n'a pas l'air bien, que dois-je faire ?     arrosage
10   combien de fois dois je donner de l'eau à ma p...     arrosage
11        comment puis je donner à boire à ma plante ?     arrosage
12          quand dois je donner à boire à ma plante ?     arrosage
13                                     arrosage 

In [ ]:
# attention voir ligne 188
# 188                        je dois mettre du terreau ?          NaN

In [3]:
y = list(data["intention"])

In [4]:
liste = data["sentence"].tolist()
j=0;

#dictionnaire des mots connus
words = list()
allowed_pos = ['VERB', 'NOUN', 'PROPN', 'ADJ']

# process sentences
for i in data["sentence"]:
    # convert all letters to lower case
    i = i.lower()
    i = i.replace('-', ' ')
    
    regex = re.compile("plante([^r]|$)")
    i = regex.sub('', i)
    
    nlp_fr = spacy.load('fr_core_news_sm')
    tokens = nlp_fr(i)

    new_sentence = ''
    for token in tokens:
        if ( allowed_pos.count(token.pos_) > 0 ):
                if (token.lemma_ != 't'): 
                    new_sentence += str(token.lemma_) + ' '
                    words.append(str(token.lemma_))
        
    liste[j] = new_sentence;
    j += 1

In [5]:
words = list(words)

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(liste).todense() #renvoie le bag of words

In [7]:
loo = LeaveOneOut()
loo.get_n_splits(X)

215

In [8]:
compt_svm = 0

for train_index, test_index in loo.split(X): 
    X_train = []
    y_train = []
    for index in train_index:
        X_train+=X[index].tolist()
        y_train.append(y[index])
    X_test = X[test_index].tolist()
    y_test = y[int(test_index)]
    
    clf_svm = svm.SVC(kernel='linear')
    clf_svm.fit(X_train, y_train)

    if( clf_svm.predict(X_test) != 'nan' ):
        if int(clf_svm.predict(X_test))==y_test:  # si prediction svm correcte
            compt_svm += 1 

In [9]:
print("Précision & Recall SVM :", compt_svm/len(X))

Précision & Recall SVM : 0.7534883720930232


In [10]:
from joblib import dump, load
dump(clf_svm, 'clf_svm.joblib')

['clf_svm.joblib']

In [11]:
# Avant de les charger à nouveau
clf_svm = load('clf_svm.joblib') 

In [12]:
# Test with a simple sentence
s = "Dois je donner de l'eau à ma plante"

# Pre-processing
s = s.lower()

regex = re.compile("plante([^r]|$)")
s = regex.sub('', s)

nlp_fr = spacy.load('fr_core_news_sm')
tokens = nlp_fr(s)

words = list()
                
# Lemmatize
for token in tokens:
    if ( allowed_pos.count(token.pos_) > 0 ):
        if (token.lemma_ != 't'): 
            words.append(str(token.lemma_))

           
j = 0;
vector = vectorizer.get_feature_names()

# Create vector
for word in vector:
    vector[j] = words.count(word);           
    j += 1



p = clf_svm.predict([vector])
print(intentions[ int(p[0]) ])

arrosage


In [39]:
# detection de la plante choisie

plants = ['basilic','Jacques','olivier','Lola']

sentence = "Dois je donner de l'eau à Lola"

In [45]:
def identifyWantedPlant(plant_list, sentence):
    # plant_list composed of the species and nicknames of the possessed plants
    sentence = sentence.lower()
    for count,name in enumerate(plant_list):
        if re.findall(name.lower(), sentence):
            # if found, returns the position in the given list
            return count
    return -1

In [46]:
# test
print(identifyWantedPlant(plants, "est-ce que jacques veut boire ???"))

1
